<a href="https://colab.research.google.com/github/lokesharma-dev/Fake-News-Detection/blob/master/Adversarial_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Execute from here**

# Virtual Adversarial Training (Embedding Noise)

In [ ]:
import numpy as np
import random
import time
#------------------- Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM, Bidirectional

word_index = 12186 # For above data, computed in some other codeblock
MAX_VOCAB_SIZE = (word_index) + 1 # maximum no of unique words
MAX_DOC_LENGTH = 500 # maximum no of words in each sentence
EMBEDDING_DIM = 300 # Embeddings dimension from Glove directory
print(tf.__version__)

sequences = np.load('/content/drive/My Drive/Colab Notebooks/datasets/data.npy')
y = np.load('/content/drive/My Drive/Colab Notebooks/datasets/label.npy')

2.2.0


In [ ]:
# Shuffle data random before splitting
indices = np.arange(sequences.shape[0])
random.Random(1).shuffle(indices)
data = sequences[indices]
labels = y[indices]

num_test_samples = int(0.2 * data.shape[0])
x_train = data[:-num_test_samples]
y_train = labels[:-num_test_samples]
x_test = data[-num_test_samples:]
y_test = labels[-num_test_samples:]
print(x_train.shape, y_train.shape)
print(x_train[0].shape, y_train[0].shape)

(400, 500) (400,)
(500,) ()


Custom Functions

<TakeDataset shapes: ((500,), ()), types: (tf.int32, tf.int64)>

In [ ]:
def compute_kld(p_logit, q_logit):
  p = tf.nn.softmax(p_logit)
  q = tf.nn.softmax(q_logit)
  kl_score = tf.reduce_sum( p * (tf.math.log(p+1e-16) - tf.math.log(q+1e-16)), axis = 1)
  return kl_score # lower kl means closer the distributions are

In [63]:

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(40, seed=0, reshuffle_each_iteration=True)
train_dataset = train_dataset.repeat(2)
batched_dataset = train_dataset.batch(40)

features, labels = next(iter(batched_dataset.take(4)))

In [60]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=MAX_VOCAB_SIZE,
                           output_dim=EMBEDDING_DIM,
                           input_length=MAX_DOC_LENGTH,
                           name='Embedding_layer'))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(2))


In [ ]:
tf.keras.utils.plot_model(model, show_layer_names=True, show_shapes=True)

In [73]:
pred = model(features[:10])
pred_prob = tf.nn.softmax(pred)
print("Prediction: {}".format(tf.argmax(pred, axis=1)))
print("    Labels: {}".format(labels[:10]))

Prediction: [1 1 1 1 1 1 1 1 1 1]
    Labels: [0 1 0 1 0 0 1 1 1 1]


In [74]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y, training):
  y_ = model(x, training=training)
  return loss_object(y_true=y, y_pred=y_)

l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))

Loss test: 0.6925274729728699


In [76]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels, training=True).numpy()))

Step: 0, Initial Loss: 0.6925274729728699
Step: 1,         Loss: 0.678128182888031


In [83]:
train_loss_results = []
train_accuracy_results = []

num_epochs = 2

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  for x, y in batched_dataset:
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    epoch_accuracy.update_state(y, model(x, training=True))

  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 0.679, Accuracy: 52.125%
Epoch 001: Loss: 0.665, Accuracy: 53.250%


In [ ]:
noise = 

seq_input = tf.convert_to_tensor((x_train[0],))
seq_input.shape

TensorShape([1, 500])

In [ ]:
v = keras.layers.Embedding(input_dim=MAX_VOCAB_SIZE,
                           output_dim=EMBEDDING_DIM,
                           name='Embedding_layer')(seq_input)
                      
h1 = layers.LSTM(units=128, name='LSTM_layer_v')(v)
p_logit = layers.Dense(units=10, name='Dense_layer_v')(h1)

In [ ]:
u

tf.Tensor(
[[[ 5.8567409e-40 -6.5531863e-40 -5.0462019e-40 ...  1.1564804e-39
   -2.1842179e-40  2.3464322e-40]
  [ 7.4412312e-40 -6.9397625e-40 -6.7859700e-40 ...  1.5409463e-39
   -2.3563674e-40  4.0652789e-40]
  [ 7.9690863e-40 -9.2688186e-40 -8.8635071e-40 ...  1.8805425e-39
   -2.4893226e-40  4.1447886e-40]
  ...
  [ 7.3493793e-05  1.1440966e-04 -2.6020425e-04 ...  2.3355914e-04
    8.3595383e-05  1.0019035e-04]
  [ 9.6998221e-05  1.4272197e-04 -3.1448196e-04 ...  2.9717124e-04
    1.0960357e-04  1.0925635e-04]
  [ 1.3966946e-04  1.8355183e-04 -3.8214438e-04 ...  4.0356658e-04
    1.3134073e-04  9.9243276e-05]]], shape=(1, 500, 300), dtype=float32)


## VAT Model

In [ ]:
input_emb = createEmbd(inputs)
noise_emb = tf.random.uniform(shape=tf.shape(input_emb)) # Idea is to add noise to these embeddings
#noise_emb = tf.math.add(input_emb, noise_emb)
noise_emb = input_emb + noise_emb

input_h1 = layers.LSTM(units=128,name="Input_h1")(input_emb)
noise_h1 = layers.LSTM(units=128,name="Noise_h1")(noise_emb)

p_logit = layers.Dense(units=16, activation='relu', name="p_logit")(input_h1)
p_logit_r = layers.Dense(units=16, activation='relu', name="p_logit_r")(noise_h1)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(noise_emb)
    kl_score = compute_kld(p_logit, p_logit_r)
    kl_score = tf.convert_to_tensor(kl_score, dtype=tf.float32)
grads = tape.gradient(kl_score, noise_emb) # Differentiate kl_score with respect to noise_embd

#p_logit = tf.stop_gradient(p_logit)
#p_logit_r = tf.stop_gradient(p_logit_r)

# Due to some reason the first execution returned "None" for gradients so manually added the shape to be able to build the model
if grads is None:
  grads = tf.random.uniform(shape=tf.shape(noise_emb)) 

vadv_emb = tf.math.add(input_emb, grads)
vadv_h1 = layers.LSTM(units=128,name="vadv_h1")(vadv_emb)
q_logit = layers.Dense(units=16, activation='relu', name="q_logit")(vadv_h1)

vat_loss = compute_kld(p_logit, q_logit) # I need to add this vat loss(Scalar) to the final cost function

# logits = layers.average([p_logit, p_logit_r, q_logit])
outputs = layers.Dense(units=1, activation='softmax', name="output")(q_logit)
model = keras.Model(inputs, outputs)

model.add_loss(vat_loss)